# Working with Callback Functions

MIP solvers are complicated combinations of many techniques: cutting planes, heuristics, branching rules, etc.

Some solvers allow you to customize aspects of the solve process in a deeper way than just setting options for these parameters. You can provide code to be run when certain events happen, and the solver **calls back** to these functions to ask what action(s) should be taken. Why might you want to do this?

* The solver is struggling to find an integer solution. You know an efficient way to take a fractional solution and convert it to a good, if not optimal, integer solution. You can put this algorithm inside a **heuristic callback** that is called whenever a new fractional solution is found.
* You have done an analysis of the structure of your MIP and have realized that you can find constraints that will cut off fractional solutions so that your LP relaxation is closer to integer points. You can write this as a **cut callback**.

The examples we look at today will demonstrate cuts that are even more critical than these two types, because it enables whole types of problems to be solved that would be very difficult otherwise. In particular, consider a problem that has a very large number of constraints, **most of which will not be binding at the optimal solution**. This suggests that we probably don't need all those constraints to provided explicitly to the solver - instead, we can provide them implicitly with a **lazy constraint/cut callback**.

## Application 1: Sparse Linear Regression

## Application 2: Robust Portfolio Optimization

Portfolio optmization is the problem of constructing a portfolio of assets to maximize returns, but usually with some consideration towards the risk of the portfolio. If we maximize for return, we will usually also have the highest chance of losing money. On the other hand, there is often a (very) low risk option that has minimal returns (e.g. US government bonds). We seek to construct optimization models that are let us explore this spectrum of options.

The "stochastic programming" approach would estimate a probability distribution from data for each asset we are considering purchasing, and then we can do things like

- minimize $StdDev[Profit]$, subject to $Exp[Profit] \geq P_{min}$
- maximize $E[Profit]$, subject to $StdDev[Profit] \leq S_{max}$

**Robust optimization** is an alternative method that, instead of saying that the uncertain return of the assets coming from probability distributions, says the returns are drawn from a bounded set of outcomes: an **uncertainty set**.

### Setting up the Problem

We will consider the following robust portfolio problem.

- Let $0 \leq x_i \leq 1$ be the share of our money we put into asset $i$.
  - We need the additional constraint then that $\mathbf{e}^\prime \mathbf{x} = 1$
  - We'll also impose a restriction that we can use no more than a quarter of the assets available.
  - Let $y_i \in \{0,1\}$, $y_i = 1 \iff x_i > 0$, and $\mathbf{e}^\prime \mathbf{y} \leq \frac{N}{4}$

- Let $p_i$ be the uncertain profit for asset $i$, with $\mathbf{p}\in U$, where...

- $U$ is out uncertainty set. By varying its size and shape of the uncertainty set $U$ we can tradeoff between expected return and the worst-case return. We will assume we have (as data)
  - $\bar{p}_i$, the expected return of each asset
  - $\sigma_i$, the standard devition of return for each asset

We will use the **ellipsoidal uncertainty set**

$$ U^\Gamma = \left\{ \mathbf{p} \mid p_i = \bar{p}_i + \sigma_i d_i, \|\mathbf{d}\|\leq \Gamma \right\}$$

*on board: diagram*

So we can write out our problem now as

$$
\max_{z, \mathbf{x}\geq \mathbf{0}} z \quad \text{subject to}\\
z \leq \mathbf{p}^\prime \mathbf{x} \quad \forall \mathbf{p} \in U \\
\mathbf{e}^\prime \mathbf{x} = 1 \\
y_i \geq x_i \\
\mathbf{e}^\prime \mathbf{y} \leq \frac{N}{4}
$$

The problem with the first constraint is that it is actually an **infinite** number of constraints - one for every possible value of $\mathbf{p}$. We conjecture though that only a small number of them are needed to a get solution that "mostly" satisifes that constraint. We'll add them lazily using **lazy constraints** in JuMP with Gurobi. 

Whenever Gurobi finds a new integer-feasible solution $\left( \mathbf{x}^\ast, \mathbf{y}^\ast, z^\ast \right)$, we will try to generate a new constraint. We do that by solving an **embedded** optimization problem:

$$CUT(\mathbf{x}^\ast) = {\arg \min}_{\mathbf{p}\in U} \mathbf{p}^\prime \mathbf{x}^\ast$$

*on board: diagram*

We'll only add this new constraint if the it'd be violated by the current solution by more than a tolerance. Today we'll actually solve this embedded problem using Gurobi, but as an **exercise** you can solve it in closed-form - see how much of an improvement in solve times you get!

In [6]:
using JuMP, Gurobi

# Generate data
n = 20
p̄ = [1.15 + i*0.05/150 for i in 1:n]
σ = [0.05/450*√(2*i*n*(n+1)) for i in 1:n]

function solve_portfolio()
    port = Model(solver=GurobiSolver())
    
    @variable(port, z ≤ maximum(p̄))
    @objective(port, Max, z)
    @variable(port, 0 ≤ x[1:n] ≤ 1)
    @constraint(port, sum(x) == 1)
    
    @variable(port, y[1:n], Bin)
    for i in 1:n
        @constraint(port, y[i] ≥ x[i])
    end
    @constraint(port, sum(y) ≤ div(n,4))
    
    # Link z to x
    function portobj(cb)
        # Get values of z and x
        zval = getValue(z)
        xval = getValue(x)[:]
    
        # Find most pessimistic value of p'x
        # over all p in the uncertainty set
        rob = Model(solver=GurobiSolver(OutputFlag=0))
        @variable(rob, p[i=1:n])
        @variable(rob, d[i=1:n])
        @objective(rob, Min, dot(xval,p))
        Γ = sqrt(10)
        @constraint(rob, sum{d[i]^2,i=1:n} ≤ Γ^2)
        for i in 1:n
            @constraint(rob, p[i] == p̄[i] + σ[i]*d[i])
        end
        solve(rob)
        worst_z = getObjectiveValue(rob)
        @show (zval, worst_z)
        worst_p = getValue(p)[:]
        
        # Is this worst_p going to change the objective
        # because worst_z is worse than the current z?
        if worst_z < zval - 1e-2
            # Yep, we've made things worse!
            # Gurobi should try to find a better portfolio now
            @lazyconstraint(cb, z ≤ dot(worst_p,x))
        end
    end
    setLazyCallback(port, portobj)
    
    solve(port)
    
    return getValue(x)[:]
end

solve_portfolio()

Optimize a model with 358 rows, 729 columns and 2950 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 358 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap -
  0.227824 seconds (114.97 k allocations: 3.850 MB)


### Exercise: Replace inner model with closed form expression

The cutting plane problem was:

$${\min}_{\mathbf{p}\in U} \mathbf{p}^\prime \mathbf{x}^\ast$$

$$ U^\Gamma = \left\{ \mathbf{p} \mid p_i = \bar{p}_i + \sigma_i d_i, \|\mathbf{d}\|\leq \Gamma \right\}$$

Lets do a little rearrangement, so instead it is

$$ U^\Gamma = \left\{ \mathbf{p} \mid \sqrt{\sum_{i=1}^n \left( \frac{p_i - \bar{p}_i}{\sigma_i} \right)} \leq \Gamma \right\}$$

So the problem is maximizing a linear function over an ellipse, which if you go through the KKT conditions you'll find has a nice closed form solution:

$$ p^\ast_i = \bar{p}_i + \frac{\Gamma}{\| diag(\sigma) \mathbf{x}^\ast \|} \sigma^2_i x^\ast_i$$

In [ ]:
# Convergence Plots

In [ ]:
# Extracting intermediate solutions